In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
from compas_surrogate.compas_output_parser.compas_output import CompasOutput
co = CompasOutput.from_h5("test_data/COMPAS_Output/")
co

,Value
Setting,
Actual-Random-Seed,1667184197
Actual-Random-Seed-Derivation,CALCULATED
COMPAS-Version,02.33.01
COMPAS-Version-Derivation,CALCULATED
Clock-Time,0.058344
...,...
zeta-adiabatic-arbitrary-Derivation,DEFAULT_USED
zeta-main-sequence,2.0
zeta-main-sequence-Derivation,DEFAULT_USED


In [19]:
import xarray as xr



{'outdir': 'test_data/COMPAS_Output/',
 'Run_Details': {'Actual-Random-Seed': 1667184197,
  'Actual-Random-Seed-Derivation': 'CALCULATED   ',
  'COMPAS-Version': '02.33.01',
  'COMPAS-Version-Derivation': 'CALCULATED   ',
  'Clock-Time': 0.058344,
  'Clock-Time-Derivation': 'CALCULATED   ',
  'Objects-Created': 5,
  'Objects-Created-Derivation': 'CALCULATED   ',
  'Objects-Requested': 5,
  'Objects-Requested-Derivation': 'CALCULATED   ',
  'PISN-lower-limit': 60.0,
  'PISN-lower-limit-Derivation': 'DEFAULT_USED ',
  'PISN-upper-limit': 135.0,
  'PISN-upper-limit-Derivation': 'DEFAULT_USED ',
  'PPI-lower-limit': 35.0,
  'PPI-lower-limit-Derivation': 'DEFAULT_USED ',
  'PPI-upper-limit': 60.0,
  'PPI-upper-limit-Derivation': 'DEFAULT_USED ',
  'Run-End': 'Mon Oct 31 13:43:17 2022\n',
  'Run-End-Derivation': 'CALCULATED   ',
  'Run-Start': 'Mon Oct 31 13:43:17 2022\n',
  'Run-Start-Derivation': 'CALCULATED   ',
  'Wall-Time': '0000:00:00',
  'Wall-Time-Derivation': 'CALCULATED   ',
  'ac

In [39]:
import datapane as dp

In [43]:
dp.DataTable(co.BSE_System_Parameters)

ImportError: pyarrow>=1.0.0 is required for PyArrow backed StringArray.

In [42]:
! pip install pyarrow

You should consider upgrading via the '/Users/avaj0001/Documents/projects/compas_dev/compas_venv/bin/python -m pip install --upgrade pip' command.
